### Youtube Sentence Pos Tag Analysis

In [19]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [20]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [21]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification
threshold_num = 2

In [22]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [23]:
def column_word_list(df, column):
    word_list = []
    for i in df[f"{column}"].dropna():
        words = i.split(",")
        for j in words:
            word = j.strip()
            word_list.append(word)
    return word_list

In [24]:
def column_word(df, column_list):
    '''
    column_word(df_pos_tag_group, df_pos_tag_group.columns) \n
    column_word(df_pos_tag_group, ["col_1","col_2"])
    '''
    word_list_all = []
    for i in column_list:
        word_list = column_word_list(df, f"{i}")
        word_list_all.append(word_list)
    return word_list_all

In [25]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    if any([True if i>count_condition else False for i in count_list]):
        return "Fail"
    else:
        return "Pass"

    #return any([True if i>count_condition else False for i in count_list])  

In [26]:
df_pos_tag_group = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/\
Data/{lang_folder.capitalize()}/{file}.xlsx", sheet_name=f"{sheets}")
df_pos_tag_group

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
0,adam,başka,al,ama,"ben, beni, benimle, bana, bence, benim",artık,aynı,dostum,"mı, mısın"
1,anne,belki,"bak, bakalım",çok,"bir, biri",asla,biraz,efendim,"misin, mi"
2,baba,bile,"biliyorum, biliyorsun, bilmiyorum, biliyor",da,"bizi, bizim, bize, biz",birkaç,büyük,hadi,mü
3,"gerçek, gerçekten",birlikte,"değil, değilim",daha,"bu, bunun, buna, bunu, burada, buraya",bugün,doğru,haydi,"mu, musun"
4,iş,böyle,"demek, de",diye,"o, onu, onları, ona, onun, onlar, onunla, orada",dakika,en,hey,nasıl
5,izin,bütün,dur,evet,"seni, seninle, senin, sen, sana",devam,güzel,"iş, işte","ne, neler"
6,kız,çünkü,"ederim, et",gibi,"size, sizi, siz",gece,harika,işte,neden
7,önemli,eğer,"gel, geldi, geliyor",hayır,"şu, şunu",gün,iki,lütfen,nerede
8,özür,fazla,git,için,NaN,hala,iyi,merhaba,NaN
9,para,gerek,ister,ile,NaN,hemen,kötü,oh,NaN


In [27]:
df_youtube_sent_select = pd.read_excel("Turkish_English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN


In [28]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [29]:
result_list = []
#for i in df_youtube_sent_select["search_string"]:
for i in range(len(df_youtube_sent_select)):
    sent = df_youtube_sent_select.loc[i,"search_string"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    words = word_tokenize(sent)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([sent,id,start,list_var])

In [30]:
df_youtube_sent_class = pd.DataFrame(result_list, columns=["search_string","video_id","start_time","classification"])
df_youtube_sent_class

,search_string,video_id,start_time,classification
0,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,1222,"[0, 2, 3, 7, 3, 5, 1, 5, 7, 1, 5, 5]"
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,381,"[2, 1, 1, 1, 2, 1, 3, 1, 2, 3, 1, 2, 8, 2]"
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,466,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]"
3,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,84,"[4, 3, 0, 2, 6, 1, 5, 4, 2, 4, 0, 5]"
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,2162,"[2, 2, 4, 3, 2, 2, 4, 3, 2, 3, 5, 0, 7, 4, 3]"
...,...,...,...,...
120259,o ne o ne,1s60RPDoaNU,5133,"[4, 8, 4, 8]"
120260,o ne ki o,K8fgPqaaJrA,1653,"[4, 8, 3, 4]"
120261,o ne ne o,bIppytTQ0aI,4250,"[4, 8, 8, 4]"
120262,o mu o mu,8cU1k2ylhF4,4062,"[4, 8, 4, 8]"


In [31]:
df_youtube_sent_class["classification_condition"] = df_youtube_sent_class["classification"].apply(lambda x: classification_condition(x, threshold_num))
df_youtube_sent_class

,search_string,video_id,start_time,classification,classification_condition
0,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,1222,"[0, 2, 3, 7, 3, 5, 1, 5, 7, 1, 5, 5]",Fail
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,381,"[2, 1, 1, 1, 2, 1, 3, 1, 2, 3, 1, 2, 8, 2]",Fail
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,466,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]",Fail
3,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,84,"[4, 3, 0, 2, 6, 1, 5, 4, 2, 4, 0, 5]",Fail
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,2162,"[2, 2, 4, 3, 2, 2, 4, 3, 2, 3, 5, 0, 7, 4, 3]",Fail
...,...,...,...,...,...
120259,o ne o ne,1s60RPDoaNU,5133,"[4, 8, 4, 8]",Pass
120260,o ne ki o,K8fgPqaaJrA,1653,"[4, 8, 3, 4]",Pass
120261,o ne ne o,bIppytTQ0aI,4250,"[4, 8, 8, 4]",Pass
120262,o mu o mu,8cU1k2ylhF4,4062,"[4, 8, 4, 8]",Pass


In [32]:
df_youtube_sent_class_select = df_youtube_sent_class[df_youtube_sent_class["classification_condition"] == "Pass"]
df_youtube_sent_class_select.reset_index(drop=True, inplace=True)
df_youtube_sent_class_select

,search_string,video_id,start_time,classification,classification_condition
0,bana biraz zaman ver anne sadece biraz zaman i...,QMEyg4HluPM,1810,"[4, 6, 5, 2, 0, 1, 6, 5, 2]",Pass
1,şeyler nasıl olduğunu gerçekten çok iyi biliyo...,zgQbOmPp9pk,2710,"[1, 8, 2, 0, 3, 6, 2, 7]",Pass
2,nasıl bir şekilde olduğunu ve neden o şekilde ...,owj1QZoYqFs,14,"[8, 4, 1, 2, 3, 8, 4, 1, 2]",Pass
3,oldu sonra bir anne sonra çok iyi bir anne old...,uFd50saMf6g,406,"[2, 5, 4, 0, 5, 3, 6, 4, 0, 2, 3]",Pass
4,mı evet böyle işte tamam bir şey yok orada bak...,Mh3bgDLKt2o,603,"[8, 3, 1, 7, 7, 4, 1, 2, 4, 2, 8]",Pass
...,...,...,...,...,...
104051,o mu o mu,lsciLyjICMk,4062,"[4, 8, 4, 8]",Pass
104052,o ne o ne,1s60RPDoaNU,5133,"[4, 8, 4, 8]",Pass
104053,o ne ki o,K8fgPqaaJrA,1653,"[4, 8, 3, 4]",Pass
104054,o ne ne o,bIppytTQ0aI,4250,"[4, 8, 8, 4]",Pass


In [33]:
df_youtube_sent_class_select_merge = pd.merge(df_youtube_sent_select, df_youtube_sent_class_select, how="right", on=["search_string","video_id","start_time"])
#df_youtube_sent_class_select_merge.drop_duplicates(inplace=True)
df_youtube_sent_class_select_merge

,search_string,start_time,end_time,sentence,video_id,video_url,classification,classification_condition
0,bana biraz zaman ver anne sadece biraz zaman i...,1810,1815,bana biraz zaman ver anne sadece biraz zaman i...,QMEyg4HluPM,https://www.youtube.com/watch?v=QMEyg4HluPM&t=...,"[4, 6, 5, 2, 0, 1, 6, 5, 2]",Pass
1,şeyler nasıl olduğunu gerçekten çok iyi biliyo...,2710,2714,yani müjdelerin o diğer şeyler nasıl olduğunu ...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[1, 8, 2, 0, 3, 6, 2, 7]",Pass
2,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[8, 4, 1, 2, 3, 8, 4, 1, 2]",Pass
3,oldu sonra bir anne sonra çok iyi bir anne old...,406,410,önce kadın sonra insan oldu sonra bir anne son...,uFd50saMf6g,https://www.youtube.com/watch?v=uFd50saMf6g&t=...,"[2, 5, 4, 0, 5, 3, 6, 4, 0, 2, 3]",Pass
4,mı evet böyle işte tamam bir şey yok orada bak...,603,609,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...,"[8, 3, 1, 7, 7, 4, 1, 2, 4, 2, 8]",Pass
...,...,...,...,...,...,...,...,...
104065,o mu o mu,4062,4064,o mu o mu,lsciLyjICMk,NaN,"[4, 8, 4, 8]",Pass
104066,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[4, 8, 4, 8]",Pass
104067,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[4, 8, 3, 4]",Pass
104068,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[4, 8, 8, 4]",Pass


In [18]:
df_youtube_sent_class_select_merge.drop_duplicates(inplace=True)

TypeError: unhashable type: 'list'

In [ ]:
df_youtube_sent_class_select.to_excel("")

In [ ]:
column_word_list(df_pos_tag_group, "col_1")

In [ ]:
column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
df_test = df_youtube_sentence_select.head(2)
df_test

In [ ]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
result_list = []
for i in df_test["search_string"]:
    words = word_tokenize(i)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([i,list_var])

In [ ]:
result_list

In [ ]:
pd.DataFrame(result_list, columns=["sentence","classification"])

In [ ]:
len(column_word_list_all)

In [ ]:
from collections import Counter

In [ ]:
result_list[0][1]

In [ ]:
Counter(result_list[0][1])

In [ ]:
any(Counter(result_list[0][1]).values())

In [ ]:
list_test = list(Counter(result_list[0][1]).values())
list_test

In [ ]:
any([True if i>3 else False for i in list_test])

In [ ]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    return any([True if i>count_condition else False for i in count_list])   

In [ ]:
list_test2 = result_list[0][1]

In [ ]:
classification_condition(list_test2, 4)